## CNN implementation

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

## 1. Load Libraries

## 2. Load Dataset

## Extracting faces from train videos

## 3. Training Visualization 

## 4. Load Weights

## 5. Data Exploration

In [ ]:
#plt.bar(range(2), [len(train_files_orig),len(train_files_fake)])

## 6. Image Sample

## 7. Pre-process the Data

### Tensor Creation

### Image Augmentation For The Fake Photos

# Data generator

In [2]:
# create and configure augmented image generator
from keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(
    rescale=1./255
    #width_shift_range=0.2,  # randomly shift images horizontally (10% of total width)
    #height_shift_range=0.2,  # randomly shift images vertically (10% of total height)
    #horizontal_flip=True,
    #vertical_flip = True,
    #shear_range=0.15,
    #zoom_range=0.2,
) # randomly flip images horizontally
train_generator = train_gen.flow_from_directory(
        # This is the target directory
        "/kaggle/input/deep-fake-gp/train",
        # All images will be resized to 150x150
        target_size=(224, 224),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary',
        #color_mode="grayscale"
        )
val_gen = ImageDataGenerator(rescale=1./255)
val_generator = val_gen.flow_from_directory(
        # This is the target directory
        "/kaggle/input/deep-fake-gp/val",
        # All images will be resized to 150x150
        target_size=(224, 224),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary',
        #color_mode="grayscale"
        )

Found 1615 images belonging to 2 classes.
Found 409 images belonging to 2 classes.


## 8. Custom Architecture

In [3]:
# !pip uninstall tensorflow
# !pip install tensorflow==1.4.0
!pip install tensorflow

# !pip uninstall keras
# !pip install keras==2.0.8
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import Sequential
from tensorflow import keras
# from tensorflow.keras import layers
# !pip install tensorflow.keras.layers.normalization
!pip install keras-layer-normalization
from tensorflow.keras.layers import BatchNormalization
from keras_layer_normalization import LayerNormalization
# from keras.layers.normalization import BatchNormalization

# from LayerNormalization import BatchNormalization
"""
    Propsoed CNN architecture.

"""

model = Sequential()

# Pamameters Initialization
input_shape = ( 224, 224, 3)
activation = 'relu'
padding = 'same'
droprate = 0.1
epsilon=0.001

model = Sequential()
model.add(BatchNormalization(input_shape=input_shape))
model.add(Conv2D(filters=16, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))


model.add(Conv2D(filters=32, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=64, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=128, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=256, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(Conv2D(filters=512, kernel_size=3, activation=activation, padding=padding))
model.add(MaxPooling2D(pool_size=2))
model.add(BatchNormalization(epsilon=epsilon))
model.add(Dropout(droprate))

model.add(GlobalAveragePooling2D())
#model.add(Flatten())
#model.add(Dense(256, kernel_initializer='glorot_normal', activation='relu'))
#model.add(Dropout(0.5))

#model.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dropout(droprate))
model.add(Dense(1, activation='sigmoid'))

model.summary() # Summary of the architecture

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
  Preparing 

## Model Compile

In [5]:
# Parameters Initialization
from tensorflow.keras.optimizers import RMSprop , Adam
# from tensorflow.python.keras.optimizers import rmsprop,SGD,Adam,Adadelta

#opt = rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='binary_crossentropy',optimizer=Adam(), metrics=['accuracy'])

## Load Data

## Train Model

### Train Model on Original Data

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping 

batch_size = 10
epochs = 20

checkpointer = ModelCheckpoint(filepath='weights.custom.best.hdf5',
                               monitor='val_loss',verbose=1, save_best_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
hist = model.fit(train_generator,
                #  initial_epoch=10,
                    epochs=epochs, verbose=1, callbacks=[checkpointer,early_stopping],
                    validation_data=val_generator,
                     )




Epoch 1/20


2023-05-01 20:36:14.004067: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


51/51 [==============================] - ETA: 0s - loss: 0.9483 - accuracy: 0.5232
Epoch 1: val_loss improved from inf to 0.74498, saving model to weights.custom.best.hdf5
51/51 [==============================] - 40s 587ms/step - loss: 0.9483 - accuracy: 0.5232 - val_loss: 0.7450 - val_accuracy: 0.5306
Epoch 2/20
51/51 [==============================] - ETA: 0s - loss: 0.6997 - accuracy: 0.5585
Epoch 2: val_loss improved from 0.74498 to 0.71929, saving model to weights.custom.best.hdf5
51/51 [==============================] - 23s 459ms/step - loss: 0.6997 - accuracy: 0.5585 - val_loss: 0.7193 - val_accuracy: 0.5306
Epoch 3/20
51/51 [==============================] - ETA: 0s - loss: 0.6741 - accuracy: 0.5963
Epoch 3: val_loss did not improve from 0.71929
51/51 [==============================] - 23s 448ms/step - loss: 0.6741 - accuracy: 0.5963 - val_loss: 0.7665 - val_accuracy: 0.5306
Epoch 4/20
51/51 [==============================] - ETA: 0s - loss: 0.6589 - accuracy: 0.6037
Epoch 4: v

### Evaluate Model on Orgiginal Data

### Model Evaluation on Augumented Data

In [7]:
model.load_weights('/kaggle/working/weights.custom.best.hdf5')

In [8]:
# model = ...  # Get model (Sequential, Functional Model, or Model subclass)
model.save('/kaggle/working/savedmodel.hdf5')

In [30]:
from PIL import Image
from tensorflow.keras.preprocessing import image

def pred(image_p):
    img_path = image_p #'/kaggle/input/gray-yellow-r-lsp/Gray VS Yellow/test/Gray/WIN_20230418_06_44_46_Pro.jpg'
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224, 3))
    # convert PIL.Image.Image type to 3D tensor with shape (96, 96, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 96, 96, 3) and return 4D tensor
    x = np.expand_dims(x, axis=0)
    x=x/255
    return model.predict(x)


In [29]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
img = load_img('/kaggle/input/deep-fake-gp/test/training_real/real_00018.jpg', target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array=img_array/255
model.predict(img_array)

1/1 [==============================] - 0s 19ms/step


array([[0.6285674]], dtype=float32)

 print(pred('/kaggle/input/deep-fake-gp/test/training_fake/mid_113_1100.jpg'))
 

In [38]:
import os 
for i in os.listdir("/kaggle/input/deep-fake-gp/test/"):
    print("==============")
    for ii in os.listdir(f'/kaggle/input/deep-fake-gp/test/{i}'):
        print(f"{ii} ",pred(f'/kaggle/input/deep-fake-gp/test/{i}/{ii}')[0])

1/1 [==============================] - 0s 49ms/step
mid_397_1111.jpg  [0.25946933]
1/1 [==============================] - 0s 38ms/step
hard_232_0001.jpg  [0.6408263]
1/1 [==============================] - 0s 33ms/step
mid_360_1100.jpg  [0.36346555]
1/1 [==============================] - 0s 28ms/step
mid_183_1111.jpg  [0.67851704]
1/1 [==============================] - 0s 27ms/step
mid_113_1100.jpg  [0.65781164]
1/1 [==============================] - 0s 28ms/step
mid_307_1101.jpg  [0.49597174]
1/1 [==============================] - 0s 27ms/step
mid_40_1111.jpg  [0.40570378]
1/1 [==============================] - 0s 28ms/step
mid_479_1111.jpg  [0.63596225]
1/1 [==============================] - 0s 26ms/step
real_01060.jpg  [0.78628725]
1/1 [==============================] - 0s 27ms/step
real_00551.jpg  [0.6616218]
1/1 [==============================] - 0s 26ms/step
real_00903.jpg  [0.59186]
1/1 [==============================] - 0s 27ms/step
real_00675.jpg  [0.61157554]
1/1 [============

In [25]:
 print(pred('/kaggle/input/deep-fake-gp/test/training_fake/mid_113_1100.jpg'))

1/1 [==============================] - 0s 22ms/step
[[1.]]


In [24]:
model.predict(test_generator)

1/1 [==============================] - 0s 211ms/step


array([[0.78628725],
       [0.6115755 ],
       [0.25946945],
       [0.40570366],
       [0.63596225],
       [0.66162163],
       [0.49597162],
       [0.36346534],
       [0.5650996 ],
       [0.6785169 ],
       [0.6285673 ],
       [0.6578115 ],
       [0.6497782 ],
       [0.6408262 ],
       [0.6411729 ],
       [0.5482806 ],
       [0.59186   ]], dtype=float32)

In [21]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_directory(
        # This is the target directory
        "/kaggle/input/deep-fake-gp/test",
        # All images will be resized to 150x150
        target_size=(224, 224),
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary',
        #color_mode="grayscale"
        )

Found 17 images belonging to 2 classes.


In [36]:
import os 
import shutil
os.mkdir("testt")
for i in os.listdir("/kaggle/input/deep-fake-gp/test/"):
    try:
        
        os.mkdir(f"/kaggle/working/testt/{i}")
    except:
        print("file exists")
    for ii in os.listdir(f'/kaggle/input/deep-fake-gp/test/{i}'):
        shutil.copy(f'/kaggle/input/deep-fake-gp/test/{i}/{ii}',f'testt/{i}/{ii}')


In [37]:
!zip -r /kaggle/working/testt.zip /kaggle/working/testt/

  adding: kaggle/working/testt/ (stored 0%)
  adding: kaggle/working/testt/training_real/ (stored 0%)
  adding: kaggle/working/testt/training_real/real_01060.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00018.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00982.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00903.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00225.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00551.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00722.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00675.jpg (deflated 0%)
  adding: kaggle/working/testt/training_real/real_00815.jpg (deflated 0%)
  adding: kaggle/working/testt/training_fake/ (stored 0%)
  adding: kaggle/working/testt/training_fake/mid_183_1111.jpg (deflated 0%)
  adding: kaggle/working/testt/training_fake/mid_360_1100.jpg (deflated 0%)
  adding: kaggle/worki